In [1]:
#Fetches the data for the current Série A

import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

# Make a request to the webpage URL and extract data from fourth table
url = 'https://www.chancedegol.com.br/br23.htm'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[7]

# Loop through the rows of the table and extract the data from the cells
data = []
for row in table.find_all('tr'):
    cols = row.find_all('td')
    cols = [col.text.strip() for col in cols]
    data.append(cols)

# Write the data to a CSV file
with open('dataBRA23raw.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerows(data)
    


In [2]:
df = pd.read_csv('dataBRA23raw.csv')
df

,Data,Mandante,Unnamed: 2,Visitante,Vitória doMandante,Empate,Vitória doVisitante
0,13/11/2022,Athletico PR,3x0,Botafogo,43.1 %,26.8 %,30.1 %
1,15/04/2023,Botafogo,2x1,São Paulo,36.9 %,27.9 %,35.2 %
2,15/04/2023,Palmeiras,2x1,Cuiabá,74.0 %,18.6 %,7.4 %
3,15/04/2023,Red Bull Bragantino,2x1,Bahia,56.7 %,23.7 %,19.6 %
4,15/04/2023,Atlético MG,1x2,Vasco,55.1 %,25.9 %,19.0 %
...,...,...,...,...,...,...,...
184,13/08/2023,Fortaleza,4x0,Santos,61.3 %,22.7 %,16.0 %
185,13/08/2023,América MG,0x1,Goiás,45.8 %,23.2 %,31.0 %
186,14/08/2023,Palmeiras,1x0,Cruzeiro,69.2 %,22.4 %,8.4 %
187,14/08/2023,Red Bull Bragantino,1x1,Vasco,65.2 %,21.8 %,12.9 %


In [3]:
# This output cleans and organizes the Data.

df.rename(columns={'Mandante':'Casa','Visitante':'Fora'}, inplace =True)
# Changes dtype to datetime in first dataframe
df['Data'] = df['Data'].str.replace('/', '-')
df['Data'] = pd.to_datetime(df['Data'], format='%d-%m-%Y')
df['Temp'] = df['Data'].dt.year
df = df.sort_values(by='Data')
df['Data'] = df['Data'].dt.strftime('%d-%m-%Y')
# Creates a new column with the for the competition name

df = df.assign(Comp='Série A')

# Separates the results of second dataframe into new columns
new = df["Unnamed: 2"].str.split("x", n = 1, expand = True)
df["GC"]= new[0] 
df["GF"]= new[1]

# Compares scores to display the results in the second dataframe
df.loc[df['GC'] == df['GF'], 'Res'] = 'E'
df.loc[df['GC'] > df['GF'], 'Res'] = 'VC' 
df.loc[df['GC'] < df['GF'], 'Res'] = 'VV'

# Organizes de columns of the sacond dataframe
new_order = ['Comp','Temp','Data','Casa','GC', 'GF','Fora','Res']
df= df.reindex(columns=new_order)


#Padronizes the names of the teams
df.replace({'América MG': 'América-MG',
            'Atlético MG': 'Atlético-MG',
            'Athletico PR': 'Athletico-PR',                   
            }, inplace=True)
df.dropna(axis=0, how='all', inplace=True)



# Atributes a pontuation to each game
df['PC'] = df.apply(lambda x: 3 if x['Res'] == 'VC' else
                              1 if x['Res'] == 'E' else 0, axis=1)
df['PF'] = df.apply(lambda x: 3 if x['Res'] == 'VV' else
                              1 if x['Res'] == 'E' else 0, axis=1)

# Atributes a number of games for each season
df['J'] = df.groupby('Temp').cumcount() + 1
# Corrects any possible issue before saving as new file
df = df.dropna()
df = df.drop(index = 0)
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,15-04-2023,Botafogo,2,1,São Paulo,VC,3,0,1
2,Série A,2023,15-04-2023,Palmeiras,2,1,Cuiabá,VC,3,0,2
3,Série A,2023,15-04-2023,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
4,Série A,2023,15-04-2023,Atlético-MG,1,2,Vasco,VV,0,3,4
5,Série A,2023,15-04-2023,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
183,Série A,2023,13-08-2023,Grêmio,2,1,Fluminense,VC,3,0,184
185,Série A,2023,13-08-2023,América-MG,0,1,Goiás,VV,0,3,185
187,Série A,2023,14-08-2023,Red Bull Bragantino,1,1,Vasco,E,1,1,186
186,Série A,2023,14-08-2023,Palmeiras,1,0,Cruzeiro,VC,3,0,187


In [4]:
#Saves the data into a CSV format
df.to_csv('dataBRA23.csv', index=False)
df.to_json('dataBRA23.json', orient='records')
df

,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J
1,Série A,2023,15-04-2023,Botafogo,2,1,São Paulo,VC,3,0,1
2,Série A,2023,15-04-2023,Palmeiras,2,1,Cuiabá,VC,3,0,2
3,Série A,2023,15-04-2023,Red Bull Bragantino,2,1,Bahia,VC,3,0,3
4,Série A,2023,15-04-2023,Atlético-MG,1,2,Vasco,VV,0,3,4
5,Série A,2023,15-04-2023,Athletico-PR,2,0,Goiás,VC,3,0,5
...,...,...,...,...,...,...,...,...,...,...,...
183,Série A,2023,13-08-2023,Grêmio,2,1,Fluminense,VC,3,0,184
185,Série A,2023,13-08-2023,América-MG,0,1,Goiás,VV,0,3,185
187,Série A,2023,14-08-2023,Red Bull Bragantino,1,1,Vasco,E,1,1,186
186,Série A,2023,14-08-2023,Palmeiras,1,0,Cruzeiro,VC,3,0,187
